In [28]:
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd


class Node:
    def __init__(self, X, y, parent=None):
        # set y-values
        self.y = y

        # set other parameter names
        self.colnames = [col for col in X.columns if col != self.y.name]
        
        # set X values
        self.X = X[self.colnames]
        
        # get unique categories/labels in y
        self.categories = set(self.y)
        
        # initialise is_leaf to False or True if only one category
        self.is_leaf = False if len(self.categories) > 1 else True
        
        # get numbers in each category
        self.numbers = [np.sum(self.y == c) for c in self.categories]
        
        # set parent
        self.parent = parent
        
        # empty list of children
        self.children = []
        
        # calculate entropy
        self.entropy = self.get_entropy(self.y)
        
        # return if already a leaf node
        if self.is_leaf:
            return
        
        # calculate the max information gain across each parameter
        self.H = {}
        Hmax = -np.inf
        for col in self.colnames:
            H = self.information_gain(col, self.entropy)
            imax = np.argmax(H)
            self.H[col] = {
                "splitvalue": self.X[col].iloc[imax],
                "Hmax": H[imax],
            }

            if H[imax] > Hmax:
                Hmax = H[imax]
                
                # set parameter with maximum information gain
                self.Hmaxparam = col

    def __str__(self):
        if self.is_leaf:
            return f"Leaf: target class='{self.classification}'"
        else:
            return f"Node:\n\tleft: {self.Hmaxparam} <= {self.H[self.Hmaxparam]['splitvalue']}\n\tright: {self.Hmaxparam} > {self.H[self.Hmaxparam]['splitvalue']}"

    def __repr__(self):
        return str(self)

    def __len__(self):
        return len(self.y)

    @property
    def depth(self):
        """
        Depth of the node, i.e., how many parents
        """
        
        depth = 0
        parent = self.parent
        while parent is not None:
            depth += 1
            parent = parent.parent
        
        return depth

    @staticmethod
    def get_entropy(y):
        """
        Calculate entropy for the node.

        Parameters
        ----------
        y: array_like
            The array of categories/labels.

        Returns
        -------
        Z: float
            Entropy of node.
        """

        # number of items for each category
        nums = [np.sum(y == c) for c in set(y)]

        # total number of points in node
        N = len(y)
    
        # calculate entropy
        Z = -np.sum([(n / N) * np.log2(n / N) for n in nums])

        return Z

    def information_gain(self, parameter, pentropy):
        """
        Determine the information gain when splitting at each
        value of a given parameter.

        Parameters
        ----------
        parameter: str
            The name of the parameter on which to calculate the
            information gain.
        pentropy: float
            Parent node entropy.
        """

        N = len(self)
        H = np.zeros(N)

        # loop over different split values
        for i, sv in enumerate(self.X[parameter]):
            # truth array for split condition
            condition = self.X[parameter] <= sv
            ncondition = ~condition

            # entropy below split
            Zbelow = self.get_entropy(self.y[condition])

            # entropy above split
            Zabove = self.get_entropy(self.y[ncondition])

            # weighted entropy for each side of node
            weightedZ = (
                (np.sum(condition) / N) * Zbelow + 
                (np.sum(ncondition) / N) * Zabove
            )

            # calculate information gain
            H[i] = pentropy - weightedZ

        return H
    
    @property
    def classification(self):
        # get most probable value for classificatoin
        idx = np.argmax(self.numbers)
        lc = list(self.categories)

        return lc[idx]

    @property
    def has_children(self):
        return True if len(self.children) > 0 else False

    def add_child(self, child):
        self.children.append(child)


def build_tree(X, yname, min_leaf_points=5, max_depth=np.inf):
    # list containing the tree
    treenodes = []

    treenodes.append(Node(X, X[yname]))
    
    # build tree
    while True:
        # find end node with maximum entropy
        endnodes = [node for node in treenodes if not node.is_leaf and not node.has_children]
        idx = np.argmax([node.entropy for node in endnodes])

        # create new node with split
        parent = endnodes[idx]

        low = parent.X[parent.Hmaxparam] <= parent.H[parent.Hmaxparam]["splitvalue"]
        high = ~low
        
        for ci in [low, high]:
            newnode = Node(
                parent.X[ci],
                parent.y[ci],
                parent=parent,
            )
            
            # set when we hit a leaf
            if len(newnode) < min_leaf_points or newnode.depth > max_depth:
                parent.is_leaf = True
                parent.children = []  # make sure children is empty
                break
            
            # add in child
            parent.add_child(newnode)
            
            # add to list of nodes
            treenodes.append(newnode)
            
        # check if all deepest nodes are leaves
        if len([node for node in treenodes if not node.has_children and not node.is_leaf]) == 0:
            break

    return treenodes

In [41]:
Xdata = {
    "x": [1.0, 3.4, 5.6, 1.2, 3.7, 5.4, 7.8],
    "y": [-2.3, 3.2, -4.5, 1.5, 3.8, 4.3, 10.5],
    "type": ["A", "B", "B", "B", "A", "A", "B"],
}

X = pd.DataFrame(Xdata)

In [88]:
tree = build_tree(X, "type", min_leaf_points=5)

In [89]:
tree

[Leaf: target class='B', Leaf: target class='A']

In [79]:
y = X["type"]

In [84]:
np.sum(y == "A")

3

In [2]:
from sklearn.datasets import load_iris

In [14]:
data = load_iris(as_frame=True, return_X_y=True)

In [25]:
X = data[0]
X["type"] = pd.Series(data[1])

In [29]:
tree = build_tree(X, "type")

In [30]:
tree

[Node:
 	left: petal length (cm) <= 1.9
 	right: petal length (cm) > 1.9,
 Leaf: target class='0',
 Node:
 	left: petal width (cm) <= 1.7
 	right: petal width (cm) > 1.7,
 Node:
 	left: petal length (cm) <= 4.9
 	right: petal length (cm) > 4.9,
 Leaf: target class='2',
 Leaf: target class='1',
 Leaf: target class='2',
 Leaf: target class='1']

In [42]:
from sklearn.tree import DecisionTreeClassifier

sktree = DecisionTreeClassifier(criterion="entropy", min_samples_leaf=4)

sktree.fit(data[0], X["type"])

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=4)

In [44]:
sktree.get_n_leaves()

3

In [41]:
tree[4].depth

2